In [2]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

<IPython.core.display.Javascript object>

In [4]:
loans = pd.read_csv("data/sba_loans_cleaned3.csv")

<IPython.core.display.Javascript object>

In [5]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520954 entries, 0 to 520953
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   State               520954 non-null  object 
 1   BankState           520954 non-null  object 
 2   ApprovalFY          520954 non-null  int64  
 3   NoEmp               520954 non-null  int64  
 4   NewExist            520954 non-null  int64  
 5   RevLineCr           520421 non-null  float64
 6   LowDoc              516749 non-null  float64
 7   DisbursementGross   520954 non-null  float64
 8   MIS_Status          520954 non-null  int64  
 9   twoDigNAICS         520954 non-null  int64  
 10  is_franchise        520954 non-null  float64
 11  bank_out_of_state   520954 non-null  int64  
 12  Term_years          520954 non-null  float64
 13  job_category        520954 non-null  float64
 14  retained_category   520954 non-null  float64
 15  UrbanRural_cleaned  514461 non-nul

<IPython.core.display.Javascript object>

In [6]:
loans.head()

,State,BankState,ApprovalFY,NoEmp,NewExist,RevLineCr,LowDoc,DisbursementGross,MIS_Status,twoDigNAICS,is_franchise,bank_out_of_state,Term_years,job_category,retained_category,UrbanRural_cleaned,Disbr_year,Disbr_Month_sin,Disbr_Month_cos,sba_pre_approv,percent_SBA,bank_size,Appv_Month_sin,Appv_Month_cos
0,NC,MN,2006,2,1,0.0,0.0,253400.0,0,44,1.0,0,13.500000,0.0,0.0,0.0,2006,0.866025,5.000000e-01,1.0,0.75,1.0,0.5,0.866025
1,AR,MS,2006,7,0,0.0,0.0,137300.0,0,72,0.0,0,10.500000,0.0,0.0,0.0,2006,1.000000,6.123234e-17,1.0,0.85,0.0,0.5,0.866025
2,CA,SD,2006,18,1,1.0,0.0,438541.0,0,61,0.0,0,6.916667,1.0,2.0,0.0,2006,0.500000,8.660254e-01,1.0,0.50,2.0,0.5,0.866025
3,FL,FL,2006,4,0,1.0,0.0,51440.0,0,23,0.0,1,7.000000,0.0,1.0,0.0,2006,0.500000,8.660254e-01,1.0,0.50,2.0,0.5,0.866025
4,LA,LA,2006,3,0,0.0,0.0,50000.0,0,53,0.0,1,5.000000,0.0,0.0,0.0,2006,0.866025,-5.000000e-01,1.0,0.85,0.0,0.5,0.866025


<IPython.core.display.Javascript object>

In [8]:
drop_cols = [
    "ApprovalFY",
    "Disbr_Month_sin",
    "Disbr_Month_cos",
    "Appv_Month_sin",
    "Appv_Month_cos",
]
loans = loans.drop(columns=drop_cols)

<IPython.core.display.Javascript object>

In [11]:
loans["twoDigNAICS"].value_counts()

44    63807
72    59831
54    53022
81    50305
23    50183
62    36677
42    32749
56    28261
45    28057
33    24883
48    18501
32    12298
53    11521
71    10583
51     9162
52     8688
31     8461
61     6011
11     4249
49     1979
21     1085
22      413
55      132
92       85
0        11
Name: twoDigNAICS, dtype: int64

<IPython.core.display.Javascript object>

In [9]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520954 entries, 0 to 520953
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   State               520954 non-null  object 
 1   BankState           520954 non-null  object 
 2   NoEmp               520954 non-null  int64  
 3   NewExist            520954 non-null  int64  
 4   RevLineCr           520421 non-null  float64
 5   LowDoc              516749 non-null  float64
 6   DisbursementGross   520954 non-null  float64
 7   MIS_Status          520954 non-null  int64  
 8   twoDigNAICS         520954 non-null  int64  
 9   is_franchise        520954 non-null  float64
 10  bank_out_of_state   520954 non-null  int64  
 11  Term_years          520954 non-null  float64
 12  job_category        520954 non-null  float64
 13  retained_category   520954 non-null  float64
 14  UrbanRural_cleaned  514461 non-null  float64
 15  Disbr_year          520954 non-nul

<IPython.core.display.Javascript object>

In [12]:
num_cols = ["NoEmp", "DisbursementGross", "Term_years", "Disbr_year", "percent_SBA"]
cat_cols = [
    'State',
    'BankState',
    'MIS_Status',
    'twoDigNAICS',
    'job_
    '
]

<IPython.core.display.Javascript object>